## Apresentação de dados usando Bokeh 
#### Biblioteca interativa de visualização para em browsers web que provê uma construção concisa, elegante e versátil de gráficos, auxiliando na contrução rápida e fácil de dashboards.

In [1]:
#!pip install bokeh

### Importação / Carga dos dados + Filtro de informações + Dashboard
(Obs a aplicação do governo muda o  endereço de acesso, por isso os dados são referentes ao dia 10/04/2020)

In [2]:

def loading_data():
    import pandas as pd
    covid_br = pd.read_csv(
        'https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/4101ebf78c503bf35ecba5545d236c76_Download_COVID19_20200410.csv',
        ';',
        encoding='iso8859')
    # Tranformação dos dados para formato de data adequado e ordenação com base na data
    covid_br['nova_data'] = pd.to_datetime(covid_br['date'],format="%d/%m/%Y")
    covid_br.sort_values('nova_data').head()
    return covid_br

def create_frames_of_data(vector_places,vector_days,dataframe):
    import pandas as pd
    from datetime import timedelta, datetime
    # Vector of states and days
    #vetor_local = vector_places
    #print('local',vetor_local)
    vetor_dias = vector_days
    covid_br = dataframe
    
    #temporary auxiliar dataframes
    first_case =  pd.DataFrame()
    df_date = pd.DataFrame()
    metric_days = pd.DataFrame()
    complete_sequence =pd.DataFrame()

    #loop for filter states in vector all rows this estates
    #Filtrando informações relativas ao estado
    for lc in vector_places:
        state = (covid_br[(covid_br['sigla'] == lc)]
    .loc[:, ['sigla','nova_data', 'date','cases_inc','cases','deaths_inc','deaths']]
    .sort_values(['nova_data']))
        complete_sequence = complete_sequence.append(state)

    # loop for each state for metric values
    for l in vector_places:
        
        res = covid_br[((covid_br['sigla'] == l) & (covid_br['cases'] == 1) & (covid_br['cases_inc'] == 1))]
        first_case = first_case.append(res)
        
        # obtem o ultimo dia a partir da consulta anterior do estado verificado (ultima data em covi_br e menor data em res= estado sendo avaliado)
        res_lday = ((covid_br['nova_data'].max() - res['nova_data'].min()).days)
        for d in vetor_dias:
            date  = (res['nova_data']+timedelta(days=d))
            #print(l)

            result = (covid_br[
                            (
                             (covid_br['sigla'] == l) & 
                             (covid_br['nova_data']==date.iloc[0])
                            )
                            ].loc[:, ['sigla','nova_data', 'date','cases_inc','cases','deaths_inc','deaths']])
            metric_days = metric_days.append(result)
        # Capture last date and value of sequence

        date  = (res['nova_data']+timedelta(days=res_lday))
        result_ld = (covid_br[
                        (
                         (covid_br['sigla'] == l) & 
                         (covid_br['nova_data']==date.iloc[0])
                        )
                        ].loc[:, ['sigla','nova_data', 'date','cases_inc','cases','deaths_inc','deaths']])
        metric_days = metric_days.append(result_ld)
    return metric_days,complete_sequence

### Criangro gráfico e apresentando resultados

def create_dashboard(metric_days,complete_sequence):
    # Bokeh libraries
    import pandas as pd
    from bokeh.plotting import figure, show,reset_output
    from bokeh.layouts import gridplot,grid,column
    from bokeh.models import CDSView, ColumnDataSource, GroupFilter,Label, LabelSet, Range1d

    from bokeh.io import output_file,output_notebook,reset_output

    # Use reset_output() between subsequent show() calls, as needed
    reset_output()

    # notebook output
    output_notebook()

    # Output to file. If need output html page enable next line.
    #output_file('CovidAnalyse.html',      title='Corona Virus - Comparativo')

    # Create a filter object for each state
    #ESSP_cds = ColumnDataSource(complete_sequence)

    sp_data = complete_sequence[complete_sequence['sigla'] == 'SP']
    es_data = complete_sequence[complete_sequence['sigla'] == 'ES']
    rj_data = complete_sequence[complete_sequence['sigla'] == 'RJ']

    # Create a ColumnDataSource object for each state
    sp_cds = ColumnDataSource(sp_data)
    es_cds = ColumnDataSource(es_data)
    rj_cds = ColumnDataSource(rj_data)

    #Metric Days (First Cases, metric days and last Cases in metric_days dataframe)
    sp_fday = metric_days[metric_days['sigla'] == 'SP']
    es_fday = metric_days[metric_days['sigla'] == 'ES']
    rj_fday = metric_days[metric_days['sigla'] == 'RJ']

    # Create a LABEL ColumnDataSource object for each state metric
    es_annot_cds = ColumnDataSource(es_fday[['nova_data','cases']])
    sp_annot_cds = ColumnDataSource(sp_fday[['nova_data','cases']])
    rj_annot_cds = ColumnDataSource(rj_fday[['nova_data','cases']])

    #Enabled diferent places 
    plot_size_and_tools = {'x_axis_type':"datetime",'plot_height': 300, 'plot_width': 300}

    plot_size_and_tools_mini = {'x_axis_type':"datetime",'plot_height': 150, 'plot_width': 150,
                            'tools':['box_select', 'reset', 'help']}


    # Move the legend to the upper left corner
    #fig2.legend.location = 'top_left'

    
    p0 = figure(title="ES Evolution cases", **plot_size_and_tools)
    p0.vbar(x='nova_data', bottom=0, top='cases_inc', 
         color='gray', width=0.85, 
         legend_label='Daily',source=es_cds)
    
    p1 = figure(title="ES Evolution Curve", **plot_size_and_tools)
    p1.line(x='nova_data', y='cases', source=es_cds, color='black')
    p1.circle(x='nova_data', y='cases', source=es_fday, color='black')

    p2 = figure(title="SP Evolution Curve", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools_mini)
    p2.line(x='nova_data', y='cases', source=sp_cds, color='red')
    p2.circle(x='nova_data', y='cases', source=sp_fday, color='black')

    p3 = figure(title="RJ Evolution Curve", x_range=p1.x_range, y_range=p1.y_range, **plot_size_and_tools_mini)
    p3.line(x='nova_data', y='cases', source=rj_cds,  color='green')
    p3.circle(x='nova_data', y='cases', source=rj_fday, color='black')
    
    


    labels_es = LabelSet(x='nova_data', y='cases', text='cases',source=es_annot_cds, render_mode='css')
    labels_sp = LabelSet(x='nova_data', y='cases', text='cases',source=sp_annot_cds, render_mode='css')
    labels_rj = LabelSet(x='nova_data', y='cases', text='cases',source=rj_annot_cds, render_mode='css')

    citation = Label(x=70, y=70, x_units='screen', y_units='screen',
                     text='Collected by Omena. 2020-04-10', render_mode='css',
                     border_line_color='black', border_line_alpha=1.0,
                     background_fill_color='white', background_fill_alpha=1.0)

    p1.add_layout(labels_es)
    p2.add_layout(labels_sp)
    p3.add_layout(labels_rj)
    p1.add_layout(citation)
    p0.add_layout(citation)

    #l = grid([[p1],[p1,p2,p3]],sizing_mode='stretch_both')
    l = grid([[p0],[p1],[p2,p3]],sizing_mode='stretch_both')
    show(l)


# Chamando funções criadas

In [3]:
covid_br = loading_data()
vetor_states = ['ES','SP','RJ']
vetor_dias = [0,9,19,29] #ultimo dia e adicionado no algoritmo

metric_days,complete_sequence = create_frames_of_data(vector_places=vetor_states,
                                                      vector_days=vetor_dias,
                                                      dataframe=covid_br)
create_dashboard(metric_days,complete_sequence)

Loading BokehJS ...

### References
https://docs.python.org/3/library/codecs.html#standard-encodings<br>
https://docs.python.org/3/library/datetime.html#datetime.date.day<br>
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html<br>
https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html
https://hub.gke.mybinder.org/user/bokeh-bokeh-notebooks-y0pylq7v/notebooks/tutorial/00%20-%20Introduction%20and%20Setup.ipynb
<br>Grid Layout<br>
https://github.com/bokeh/bokeh/blob/1.3.2/examples/howto/layouts/dashboard.py